# AoC 2017

Solutions to 2017 [Advent of Code](http://adventofcode.com/2017/)
Based on [Peter Norvig's AoC 2016 notebook](http://nbviewer.jupyter.org/url/norvig.com/ipython/Advent%20of%20Code.ipynb)

In [44]:
# Python 3.x
import re
import numpy as np
import math
import string
import urllib.request

from collections import Counter, defaultdict, namedtuple, deque
from functools   import lru_cache
from itertools   import permutations, combinations, chain, cycle, product, islice
from heapq       import heappop, heappush

def Input(day):
    "Open this day's input file."
    filename = '{}.txt'.format(day)
    try:
        return open(filename)
    except FileNotFoundError:
        return urllib.request.urlopen("https://raw.githubusercontent.com/Bklyn/adventofcode/master/2017/" + filename)

def transpose(matrix): return zip(*matrix)

def first(iterable): return next(iter(iterable))

def nth(iterable, n, default=None):
    "Returns the nth item of iterable, or a default value"
    return next(islice(iterable, n, None), default)

# 2-D points implemented using (x, y) tuples
def X(point): return point[0]
def Y(point): return point[1]

def neighbors4(point): 
    "The four neighbors (without diagonals)."
    x, y = point
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1))

def neighbors8(point): 
    "The eight neighbors (with diagonals)."
    x, y = point 
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
            (x+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))

def cityblock_distance(p, q=(0, 0)): 
    "City block distance between two points."
    return abs(X(p) - X(q)) + abs(Y(p) - Y(q))

# Day 1: Inverse Captcha

In [45]:
def captcha(msg, halfway_around=False):
    prev = None
    answer = 0
    digits = [int(x) for x in msg if x in string.digits]
    if halfway_around:
        offset = len(digits)//2
    else:
        offset = 1
    for i in range (len (digits)):
        j = (i + offset) % len (digits)
        if digits[i] == digits[j]:
            answer += digits[i]
    return answer

assert captcha('1122') == 3
assert captcha('1111') == 4

INPUT = Input(1).read ().decode('ascii')

print (captcha(INPUT))

def captcha2(msg):
    return captcha(msg, True)

assert captcha2('1212') == 6
assert captcha2('1221') == 0
assert captcha2('123425') == 4
assert captcha2('123123') == 12
assert captcha2('12131415') == 4

print (captcha2(INPUT))

1102
1076


# [Day 2](http://adventofcode.com/2017/day/2): Corruption Checksum

In [46]:
def checksum(x):
    result = 0
    for line in x.split('\n'):
        row = [int(x) for x in line.split()]
        if not len(row):
            continue
        row = sorted (row)
        result += row[-1] - row[0]
        pass
    return result

assert checksum('''5 1 9 5
7 5 3
2 4 6 8''') == 18

INPUT = Input(2).read().decode('ascii')
print (checksum(INPUT))

def evenly(x):
    result = 0
    for line in x.split('\n'):
        row = [int(x) for x in line.split()]
        if not len(row):
            continue
        # Input is small enough, just use brute force
        row = sorted (row)
        for i, j in product (range (0, len(row)), range (1, len(row))):
            if j <= i:
                continue
            if row[j] % row[i] == 0:
                result += row[j] // row[i]
                break
    return result

assert evenly('''
5 9 2 8
9 4 7 3
3 8 6 5''') == 9

print (evenly(INPUT))

39126
258


# [Day 3](http://adventofcode.com/2017/day/3) Spiral Memory
The memory layout is an [Ulam spiral](https://en.wikipedia.org/wiki/Ulam_spiral) and I had to do some Googling to get help modeling the coordinate system.  [This StackExchange post](https://math.stackexchange.com/questions/617574/inverse-of-ulams-spiral) was instructive, though the accepted answer is actually wrong in several cases.  The code below is inspired by this answer.

In [ ]:
def coord(n):
    k = int (math.sqrt (n))
    m = n - k*k
    if k % 2 == 0:              # Even
       c = (1-(k//2), k//2)
       if m == 0:
           pass
       elif m <= k+1:
           c = (c[0]-1, c[1]-(m-1))
       else:
           m = n - (k*k + k + 1)
           c = (-k//2 + m, -k//2)
    else:                        # Odd
        c = ((k-1)//2, -(k-1)//2)
        if m == 0:
            pass
        elif m <= k+1:
            c = (c[0]+1,c[1]+m-1)
        else:
            m = n - (k*k + k + 1)
            c = (1+(k//2), 1+(k//2))
            c = (c[0]-m, c[1])
    # print (n, k, m, c)
    return c

def test_coord():
    assert coord(1) == (0,0)
    assert coord(2) == (1,0)
    assert coord(3) == (1,1)
    assert coord(4) == (0,1)
    assert coord(5) == (-1,1)
    assert coord(6) == (-1,0)
    assert coord(7) == (-1,-1)
    assert coord(8) == (0,-1)
    assert coord(9) == (1,-1)
    assert coord(10) == (2,-1)
    assert coord(11) == (2,0)
    assert coord(36) == (-2, 3)
    assert coord(37) == (-3, 3)
    assert coord(35) == (-1, 3)
    assert coord(42) == (-3, -2)

print (cityblock_distance (coord (265149)))

def part2(goal):
    cache = {(0, 0): 1}
    result = 1
    i = 1
    while result < goal:
        result = 0
        c = coord(i)
        for p in neighbors8(c):
            v = cache.get (p)
            if v is None:
                continue
            result += v
        print (i, c, result)
        cache[c] = result
        i += 1
    return result

print (part2 (265149))
